In [1]:
import sys
sys.path.append('..')
from iltransr.models.il_smiles_temperature import*

[04:19:20] ../src/storage/storage.cc:199: Using Pooled (Naive) StorageManager for CPU
[04:19:22] ../src/storage/storage.cc:199: Using Pooled (Naive) StorageManager for GPU


In [2]:
dropout = 0.05
batch_size = 128

In [3]:
net = ILNet(dropout=dropout)
net.IL_encoder = model.encoder
net.IL_src_embed =  model.src_embed
net.hybridize()
print(net)

ILNet(
  (IL_src_embed): HybridSequential(
    (0): Embedding(72 -> 128, float32)
    (1): Dropout(p = 0.0, axes=())
  )
  (IL_encoder): TransformerEncoder(
    (layer_norm): LayerNorm(eps=1e-05, axis=-1, center=True, scale=True, in_channels=128)
    (transformer_cells): HybridSequential(
      (0): TransformerEncoderCell(
        (attention_cell): MultiHeadAttentionCell(
          (_base_cell): DotProductAttentionCell(
            (_dropout_layer): Dropout(p = 0, axes=())
          )
          (proj_query): Dense(128 -> 128, linear)
          (proj_key): Dense(128 -> 128, linear)
          (proj_value): Dense(128 -> 128, linear)
        )
        (proj): Dense(128 -> 128, linear)
        (ffn): PositionwiseFFN(
          (ffn_1): Dense(128 -> 1024, linear)
          (activation): Activation(relu)
          (ffn_2): Dense(1024 -> 128, linear)
          (layer_norm): LayerNorm(eps=1e-05, axis=-1, center=True, scale=True, in_channels=128)
        )
        (layer_norm): LayerNorm(eps=1e-

In [5]:
import pandas as pd
viscosity_database = pd.read_excel(os.path.join(root_path,'datasets/il properties/viscosity_P.xlsx'),sheet_name='viscosity')

In [7]:
train_IL_smiles = viscosity_database['IL SMILES'].map(canonical_smile).map(no_split).map(preprocess)
train_T =viscosity_database['normalized_T']
train_viscosity = viscosity_database['lnη']
train_dataset = gluon.data.SimpleDataset(gluon.data.ArrayDataset(train_IL_smiles,train_T,train_viscosity))
train_dataloader= get_train_dataloader(train_dataset,batch_size=batch_size)

In [ ]:
epochs, learning_rate = 150,0.001                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
max_len = 100
save_name = 'viscosity_best.params'
train(net, train_dataloader, batch_size, learning_rate, ctx, epochs,save_name=save_name)